In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
from glob import glob
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

import json
import cv2


In [4]:
ROOT_DIR = '/scratche/users/sansiddh/DeepLesion/'
IMG_DIR = ROOT_DIR+'Images_png/'

In [16]:
df_metadata = pd.read_csv(ROOT_DIR+'DL_info.csv')


In [17]:
df_metadata


,File_name,Patient_index,Study_index,Series_ID,Key_slice_index,Measurement_coordinates,Bounding_boxes,Lesion_diameters_Pixel_,Normalized_lesion_location,Coarse_lesion_type,Possibly_noisy,Slice_range,Spacing_mm_px_,Image_size,DICOM_windows,Patient_gender,Patient_age,Train_Val_Test
0,000001_01_01_109.png,1,1,1,109,"233.537, 95.0204, 234.057, 106.977, 231.169, 1...","226.169, 90.0204, 241.252, 111.977","11.9677, 5.10387","0.44666, 0.283794, 0.434454",3,0,"103, 115","0.488281, 0.488281, 5","512, 512","-175, 275",F,62.0,3
1,000001_02_01_014.png,1,2,1,14,"224.826, 289.296, 224.016, 305.294, 222.396, 2...","217.396, 284.296, 233.978, 310.294","16.019, 6.61971","0.431015, 0.485238, 0.340745",3,0,"8, 23","0.314453, 0.314453, 5","512, 512","-175, 275",F,72.0,3
2,000001_02_01_017.png,1,2,1,17,"272.323, 320.763, 246.522, 263.371, 234.412, 3...","229.412, 258.371, 285.221, 325.763","62.9245, 48.9929","0.492691, 0.503106, 0.351754",3,0,"8, 23","0.314453, 0.314453, 5","512, 512","-175, 275",F,72.0,3
3,000001_03_01_088.png,1,3,1,88,"257.759, 157.618, 260.018, 133.524, 251.735, 1...","246.735, 128.524, 270.288, 162.618","24.1998, 13.6123","0.498999, 0.278924, 0.452792",3,0,"58, 118","0.732422, 0.732422, 1","512, 512","-175, 275",F,73.0,3
4,000001_04_01_017.png,1,4,1,17,"304.019, 230.585, 292.217, 211.789, 304.456, 2...","287.217, 206.789, 309.456, 235.585","22.1937, 9.8065","0.572678, 0.42336, 0.445674",3,0,"11, 23","0.666016, 0.666016, 5","512, 512","-175, 275",F,73.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32730,004458_01_01_059.png,4458,1,1,59,"395.737, 284.142, 385.026, 310.221, 402.198, 3...","376.707, 279.142, 407.198, 315.221","28.1937, 22.1522","0.862386, 0.522347, 0.623233",-1,0,"40, 85","0.857422, 0.857422, 5","512, 512","-175, 275",F,66.0,1
32731,004458_01_01_049.png,4458,1,1,49,"411.677, 296.13, 380.941, 321.278, 404.527, 32...","375.941, 291.13, 416.677, 328.028","39.7138, 30.688","0.845748, 0.557232, 0.570726",-1,0,"40, 85","0.857422, 0.857422, 5","512, 512","-175, 275",F,66.0,1
32732,004458_01_01_047.png,4458,1,1,47,"204.055, 307.341, 170.022, 316.047, 180.311, 3...","165.022, 297.592, 209.055, 329.357","35.1289, 22.4662","0.339313, 0.588705, 0.56127",-1,0,"40, 85","0.857422, 0.857422, 5","512, 512","-175, 275",F,66.0,1
32733,004458_01_01_102.png,4458,1,1,102,"276.457, 361.216, 220.572, 354.23, 247.117, 35...","215.572, 347.367, 281.457, 373.201","56.3199, 15.9573","0.477197, 0.730823, 0.813426",-1,0,"96, 108","0.857422, 0.857422, 5","512, 512","-175, 275",F,66.0,1


In [28]:
# The following code calculates the number of unique windowing windows per CT scan. 
# It then creates a list of that number for all CT scans. 
# Now, as you can see with the output of np.unique(windows_per_ct), ALL CTs have only 1 type of windowing. 
# So the windowing strategy of any 1 image can be safely applied to the whole CT scan. 

grouped = df_metadata.groupby(['Patient_index', 'Study_index', 'Series_ID'])
windows_per_ct = []
for name, group in grouped:
    num_windows_per_ct = len(np.unique(group['DICOM_windows'].to_numpy()))
    windows_per_ct.append(num_windows_per_ct)

np.unique(windows_per_ct)


array([ True])

In [10]:
# Total number of lesions
df_metadata.shape[0]

32735

In [11]:
# Total number of images with lesions
df_metadata.groupby('File_name').count().shape[0]

32120

In [12]:
# Converting the list columns in the dataframe from str to list
columns = ['Measurement_coordinates', 'Bounding_boxes', 'Lesion_diameters_Pixel_', 'Normalized_lesion_location', 
           'Slice_range', 'Spacing_mm_px_', 'Image_size', 'DICOM_windows']
for colname in columns:
    df_metadata[colname] = df_metadata[colname].apply(lambda x : list(map(float, x.split(', '))))

In [30]:
df_temp = df_metadata.groupby(['Patient_index', 'Study_index', 'Series_ID']).count()
df_temp

File_name  Key_slice_index  \
Patient_index Study_index Series_ID                               
1             1           1                  1                1   
              2           1                  2                2   
              3           1                  1                1   
              4           1                  1                1   
2             1           1                  2                2   
...                                        ...              ...   
4455          1           2                  1                1   
4456          1           1                  1                1   
4457          1           1                  1                1   
4458          1           1                 10               10   
4459          1           1                  1                1   

                                     Measurement_coordinates  Bounding_boxes  \
Patient_index Study_index Series_ID                                            
1             1           1                                1               1   
              2           1                                2               2   
              3           1                                1               1   
              4           1                                1               1   
2             1           1                                2               2   
...                                                      ...             ...   
4455          1           2                                1               1   
4456          1           1                                1               1   
4457          1           1                                1               1   
4458          1           1                               10              10   
4459          1           1                                1               1   

                                     Lesion_diameters_Pixel_  \
Patient_index Study_index Series_ID                            
1             1           1                                1   
              2           1                                2   
              3           1                                1   
              4           1                                1   
2             1           1                                2   
...                                                      ...   
4455          1           2                                1   
4456          1           1                                1   
4457          1           1                                1   
4458          1           1                               10   
4459          1           1                                1   

                                     Normalized_lesion_location  \
Patient_index Study_index Series_ID                               
1             1           1                                   1   
              2           1                                   2   
              3           1                                   1   
              4           1                                   1   
2             1           1                                   2   
...                                                         ...   
4455          1           2                                   1   
4456          1           1                                   1   
4457          1           1                                   1   
4458          1           1                                  10   
4459          1           1                                   1   

                                     Coarse_lesion_type  Possibly_noisy  \
Patient_index Study_index Series_ID                                       
1             1           1                           1               1   
              2           1                           2               2   
              3           1                           1               1   
              4           1                 

In [31]:
# Total number of CT scans with at least 1 lesion
df_temp.shape[0]

14601

In [32]:
# Total number of CT scans
ct_scans = glob(IMG_DIR+'*/')
len(ct_scans)

14601

In [ ]:
# Total number of CT scans matches the total number of CT scans with at least 1 lesion. 
# That means all CT scans have at least 1 lesion. So the DL_info.csv is exhaustive in its coverage.